# 랭체인 사전 준비

In [ ]:
# 패키지 설치
!pip install langchain==0.0.181
!pip install openai

In [ ]:
# 환경변수 준비
import os
os.environ["OPENAI_API_KEY"] = "<OpenAI_API의 API키>"

In [ ]:
# 패키지 설치
!pip install faiss-gpu
!pip install tiktoken

# 제네릭 체인
## LLMChain

In [ ]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate


# 템플릿 생성
template = """Q: {question}
A:"""

# 프롬프트 템플릿 생성
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

# LLMChain 생성
llm_chain = LLMChain(
    llm=OpenAI(temperature=0),
    prompt=prompt,
    verbose=True
)

# LLMChain 실행
question = "기타를 잘 치는 방법은?"
print(llm_chain.predict(question=question))

In [ ]:
# 템플릿 생성
template = """{subject}를 주제로 {target}를 작성해 주세요."""

# 프롬프트 템플릿 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["subject", "target"]
)

# LLMChain 생성
llm_chain = LLMChain(
    llm=OpenAI(temperature=0),
    prompt=prompt,
    verbose=True
)

# LLMChain 실행
print(llm_chain.predict(subject="고양이", target="시"))

## SimpleSequentialChain

In [ ]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate


# 템플릿 준비
template = """당신은 극작가입니다. 연극 제목이 주어졌을 때, 그 줄거리를 작성하는 것이 당신의 임무입니다.

제목:{title}
시놉시스:"""

# 프롬프트 템플릿 준비
prompt = PromptTemplate(
    input_variables=["title"],
    template=template
)

# LLMChain 준비
chain1 = LLMChain(
    llm=OpenAI(temperature=0),
    prompt=prompt
)

In [ ]:
# 템플릿 생성
template = """당신은 연극 평론가입니다. 연극의 시놉시스가 주어지면 그 리뷰를 작성하는 것이 당신의 임무입니다.

시놉시스:
{synopsis}
리뷰:"""

# 프롬프트 템플릿 준비
prompt = PromptTemplate(
    input_variables=["synopsis"],
    template=template
)

# LLMChain 준비
chain2 = LLMChain(
    llm=OpenAI(temperature=0),
    prompt=prompt
)

In [ ]:
from langchain.chains import SimpleSequentialChain

# SimpleSequentialChain으로 두 개의 체인을 연결
overall_chain = SimpleSequentialChain(
    chains=[chain1, chain2],
    verbose=True
)

In [ ]:
# SimpleSequentialChain 실행
print(overall_chain.run("서울 랩소디"))

## SequentialChain

In [ ]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate


# 템플릿 생성
template = """당신은 극작가입니다. 극의 제목과 시대적 배경이 주어졌을 때, 그 줄거리를 작성하는 것이 당신의 임무입니다.

제목:{title}
시대:{era}
시놉시스:"""

# 프롬프트 템플릿 생성
prompt = PromptTemplate(
    input_variables=["title", "era"], 
    template=template
)

# LLMChain 생성
chain1 = LLMChain(
    llm=OpenAI(temperature=0),
    prompt=prompt, 
    output_key="synopsis"
)

In [ ]:
# 템플릿 생성
template = """당신은 연극 평론가입니다. 연극의 시놉시스가 주어지면 그 리뷰를 작성하는 것이 당신의 임무입니다.

시놉시스:
{synopsis}
리뷰:"""

# 프롬프트 템플릿 생성
prompt = PromptTemplate(
    input_variables=["synopsis"], 
    template=template
)

# LLMChain 준비
chain2 = LLMChain(
    llm=OpenAI(temperature=0),
    prompt=prompt, 
    output_key="review"
)

In [ ]:
from langchain.chains import SequentialChain

# SequentialChain으로 두 개의 체인을 연결
overall_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["title", "era"],
    output_variables=["synopsis", "review"],
    verbose=True
)

In [ ]:
# SequentialChain 실행
print(overall_chain({"title":"서울 랩소디", "era": "100년 후의 미래"}))

# 인덱스 체인
## RetrievalQA

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# 문서 불러오기(현재 폴더에 문서를 넣어둡니다)
with open("akazukin_all.txt") as f:
    test_all = f.read()

# 청크 분할
text_splitter = CharacterTextSplitter(
    separator = "\n\n", # 구분 기호
    chunk_size=300, # 청크의 최대 문자 수
    chunk_overlap=20 # 겹치는 최대 문자 수
)
texts = text_splitter.split_text(test_all)

# 확인
print(len(texts))
for text in texts:
    print(text[:10], ":", len(text))

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

# 벡터 데이터베이스 생성
docsearch = FAISS.from_texts(
    texts=texts, # 청크 배열
    embedding=OpenAIEmbeddings() # 임베딩
)

In [ ]:
from langchain.chains import RetrievalQA

# 질의응답 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0), # LLM
    chain_type="stuff", # 체인 종류
    retriever=docsearch.as_retriever(), # 리트리버
)

In [ ]:
# 질의응답 체인 실행
print(qa_chain.run("미코의 소꿉친구 이름은?"))

## RetrievalQAWithSourcesChain

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# 문서 로드
with open("akazukin_all.txt") as f:
    test_all = f.read()

# 청크 분할
text_splitter = CharacterTextSplitter(
    separator = "\n\n", # 구분 기호
    chunk_size=300, # 청크의 최대 문자 수
    chunk_overlap=20, # 최대 오버랩 문자 수
)
texts = text_splitter.split_text(test_all)

# 확인
print(len(texts))
for text in texts:
    print(text[:10], ":", len(text))

In [ ]:
# 메타데이터 준비
metadatas=[
    {"source": "1장"},
    {"source": "2장"},
    {"source": "3장"},
    {"source": "4장"},
    {"source": "5~6장"},
    {"source": "7장"}
]

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

# 벡터 데이터베이스 생성
docsearch = FAISS.from_texts(
    texts=texts, # 청크 배열
    embedding=OpenAIEmbeddings(), # 임베딩
    metadatas=metadatas # 메타데이터
)

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain

# 소스가 있는 질의응답 체인 생성
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=OpenAI(temperature=0),
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
)

In [ ]:
# 소스가 있는 질의응답 체인 실행
print(qa({"question": "미코의 소꿉친구 이름은?"}))

# SummarizeChain

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# 문서 로드
with open("akazukin_all.txt") as f:
    test_all = f.read()

# 청크 분할
text_splitter = CharacterTextSplitter(
    separator = "\n\n", # 구분 기호
    chunk_size=300, # 청크의 최대 문자 수
    chunk_overlap=20, # 최대 오버랩 문자 수
)
texts = text_splitter.split_text(test_all)

# 확인
print(len(texts))
for text in texts:
    print(text[:10], ":", len(text))

In [ ]:
from langchain.docstore.document import Document

# 청크 배열을 문서 배열로 변환
docs = [Document(page_content=t) for t in texts]

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import OpenAI

# 요약 체인 생성
chain = load_summarize_chain(
    llm=OpenAI(temperature=0),
    chain_type="map_reduce",
)

In [ ]:
# 요약 체인 실행
chain.run(docs)


# 유틸리티 체인
## PALChain


In [ ]:
from langchain.chains import PALChain
from langchain import OpenAI

# PALChain 생성
pal_chain = PALChain.from_math_prompt(
    llm=OpenAI(), 
    verbose=True
)

# PALChain 실행
question = "제인은 앨리스가 키우는 반려동물의 3배가 되는 반려동물을 키우고 있다. 앨리스가 2마리의 반려동물을 키우고 있다면 두 사람이 키우고 있는 반려동물의 총 마리 수는?"
print(pal_chain.run(question))

## OpenAIModerationChain

In [ ]:
from langchain.chains import OpenAIModerationChain

# OpenAIModerationChain 준비
chain = OpenAIModerationChain()

In [ ]:
# 문제없는 발언
chain.run("This is OK!")

In [ ]:
# 문제 발언
chain.run("I'll kill you!")

In [ ]:
from langchain.chains import OpenAIModerationChain

# OpenAIModerationChain 준비
chain = OpenAIModerationChain(error=True)

try:
    # 문제 있는 발언
    chain.run("I'll kill you!")
except ValueError as e:
    print("문제 발언입니다!")
    print(e)